In [12]:
%matplotlib inline

import os
import glob
import numpy as np
from numpy.fft import fft, ifft, fftfreq
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation
import seaborn as sns
from nltools.data import Brain_Data, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import isc, isfc, isps, fdr, threshold, phase_randomize, circle_shift, _butter_bandpass_filter, _phase_mean_angle, _phase_vector_length
from nilearn.plotting import view_img_on_surf, view_img
from sklearn.metrics import pairwise_distances
from sklearn.utils import check_random_state
from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel
from scipy.signal import hilbert
import networkx as nx
from IPython.display import HTML

import subprocess
import pickle
from datetime import datetime
from time import time
import pdb

In [2]:
def adillyofapickle(basepath,dic, name):
    datefmt='%m-%d-%Y_%I-%M-%S'
    st = datetime.fromtimestamp(time()).strftime(datefmt)
    if os.path.exists(os.path.join(basepath,'tmp')):
        print('already have tmp')
    else:
        os.makedirs(os.path.join(basepath,'tmp'))
    pickle.dump(dic, open(os.path.join(basepath,'tmp','%s_%s'%(name,st)), 'wb'), protocol=4)


In [3]:
def onetoughjar(p):
    list_of_files = glob.glob(p) # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getmtime)
    with open(latest_file, 'rb') as pickle_file:
        try:
            while True:
                output = pickle.load(pickle_file)
        except EOFError:
            pass
    return(output)

In [4]:
def get_subject_roi(data_dict, roi_num):
    sub_rois = {}
    for sub, value in data_dict.items():
        sub_rois[sub] = value[ROIS['0'].iloc[10]]
    return pd.DataFrame(sub_rois)

## Re runs
For some reason 116 and 117 wouldn't work on renci. Ran locally below

In [13]:
tim = onetoughjar(os.path.join(datapath,'time*'))

In [19]:
list(tim['BRO']['run-1']['sub-047'].columns)

['Frontal_Pole/label',
 'Insular_Cortex/label',
 'Superior_Frontal',
 'Middle_Frontal',
 'Inferior_Frontal',
 'Inferior_Frontal_5',
 'Precentral_Gyrus/label',
 'Temporal_Pole/label',
 'Superior_Temporal',
 'Superior_Temporal_9',
 'Middle_Temporal',
 'Middle_Temporal_11',
 'Middle_Temporal_12',
 'Inferior_Temporal',
 'Inferior_Temporal_14',
 'Inferior_Temporal_15',
 'Postcentral_Gyrus/label',
 'Superior_Parietal',
 'Supramarginal_Gyrus,',
 'Supramarginal_Gyrus,_19',
 'Angular_Gyrus/label',
 'Lateral_Occipital',
 'Lateral_Occipital_22',
 'Intracalcarine_Cortex/label',
 'Frontal_Medial',
 'Juxtapositional_Lobule',
 'Subcallosal_Cortex/label',
 'Paracingulate_Gyrus/label',
 'Cingulate_Gyrus,',
 'Cingulate_Gyrus,_29',
 'Precuneous_Cortex/label',
 'Cuneal_Cortex/label',
 'Frontal_Orbital',
 'Parahippocampal_Gyrus,',
 'Parahippocampal_Gyrus,_34',
 'Lingual_Gyrus/label',
 'Temporal_Fusiform',
 'Temporal_Fusiform_37',
 'Temporal_Occipital',
 'Occipital_Fusiform',
 'Frontal_Operculum',
 'Central

In [ ]:
# test = get_subject_roi(tim['BRO']['run-1'], 116)

In [ ]:
# # condition = 'BRO'
# # ROI = 117
# df = pd.concat([get_subject_roi(tim[condition]['run-1'], ROI), get_subject_roi(tim[condition]['run-2'], ROI)], ignore_index=True)
# inp = df.dropna(axis=1)
# stats = isc(inp, n_bootstraps=5000, metric='median', method='%s'%('circle_shift'), return_bootstraps = True)
# packit = {'%s'%condition:{'%s_ROI'%ROI:{'stats':stats, 'df':inp,'method':'circle_shift'}}}

# adillyofapickle(datapath,packit, '%s_%s_%s'%(condition,ROI,'circle_shift'))


In [5]:
homepath = '/Users/gracer/OneDrive - University of Wyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/BRO/scripts'
datapath = '/Users/gracer/OneDrive - University of Wyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/BRO/data/BRO_isc/tmp'
maskpath = '/Users/gracer/OneDrive - University of Wyoming/0. Lab/M2AENAD Lab - Documents/RESEARCH/BRO/data/atlas'

In [7]:
ROIS = pd.read_csv(os.path.join(datapath,'ROIS.csv'))

In [8]:
results_circle = {}
for x in range(0,118):
    PLA = 'PLA_%s_circle*'%x
    BRO = 'BRO_%s_circle*'%x
    try:
        results_circle.update({
         x:{'PLA':onetoughjar(os.path.join(datapath,PLA)),
            'BRO': onetoughjar(os.path.join(datapath,BRO))}})    
    except ValueError:
        print(x)

                    

In [9]:
results_circle.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117])

## make dataframe

In [10]:
d = {'cond': [], 'ROI': [], 'isc':[], 'p':[], 'ci-':[],'ci+':[]}
for ROI, item in results_circle.items():
    for cond, data in item.items():
        d['ROI'].append(ROI)
        d['cond'].append(cond)
        d['isc'].append(data[cond]['%s_ROI'%ROI]['stats']['isc'])
        d['p'].append(data[cond]['%s_ROI'%ROI]['stats']['p'])
        d['ci-'].append(data[cond]['%s_ROI'%ROI]['stats']['ci'][0])
        d['ci+'].append(data[cond]['%s_ROI'%ROI]['stats']['ci'][1])
df = pd.DataFrame(data=d)

In [11]:
df.to_csv(os.path.join(datapath,'circle.csv'), sep = ',')

In [ ]:
ttest_rel(rvs1, rvs3)